In [26]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

In [28]:
transactions = pd.read_csv('Transactions.csv')
products = pd.read_csv('Products.csv')
customers = pd.read_csv('Customers.csv')

Merge dataset

In [29]:
transactions_with_products = transactions.merge(products, on="ProductID")
customer_transactions = transactions_with_products.merge(customers, on="CustomerID")

In [30]:
customer_transactions.drop(columns=['Price_y'], inplace=True)

In [31]:
customer_transactions.rename(columns={'Price_x': 'Price'}, inplace=True)


In [32]:
customer_transactions.head(3)

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,ProductName,Category,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,Andrea Jenkins,Europe,2022-12-03
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,Brittany Harvey,Asia,2024-09-04
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,Kathryn Stevens,Europe,2024-04-04


Aggregating customer level data

In [33]:
customer_profiles = customer_transactions.groupby("CustomerID").agg(
    total_spent=("TotalValue", "sum"),
    total_quantity=("Quantity", "sum"),
    avg_price=("Price", "mean"),
    region=("Region", "first"),
    category_counts=("Category", lambda x: x.value_counts().to_dict())
).reset_index()

In [34]:
customer_profiles

,CustomerID,total_spent,total_quantity,avg_price,region,category_counts
0,C0001,3354.52,12,278.334000,South America,"{'Electronics': 3, 'Books': 1, 'Home Decor': 1}"
1,C0002,1862.74,10,208.920000,Asia,"{'Home Decor': 2, 'Clothing': 2}"
2,C0003,2725.38,14,195.707500,South America,"{'Home Decor': 2, 'Clothing': 1, 'Electronics'..."
3,C0004,5354.88,23,240.636250,South America,"{'Books': 3, 'Home Decor': 3, 'Electronics': 2}"
4,C0005,2034.24,7,291.603333,Asia,"{'Electronics': 2, 'Home Decor': 1}"
...,...,...,...,...,...,...
194,C0196,4982.88,12,416.992500,Europe,"{'Home Decor': 2, 'Books': 1, 'Clothing': 1}"
195,C0197,1928.65,9,227.056667,Europe,"{'Electronics': 2, 'Home Decor': 1}"
196,C0198,931.83,3,239.705000,Europe,"{'Electronics': 1, 'Clothing': 1}"
197,C0199,1979.28,9,250.610000,Europe,"{'Electronics': 2, 'Home Decor': 2}"


One-hot encode for Region

In [35]:
encoder = OneHotEncoder()
encoded_regions = encoder.fit_transform(customer_profiles[["region"]]).toarray()
encoded_regions_df = pd.DataFrame(
    encoded_regions, columns=[f"region_{cat}" for cat in encoder.categories_[0]]
)


In [36]:
customer_profiles = pd.concat([customer_profiles, encoded_regions_df], axis=1)
customer_profiles.drop(columns="region", inplace=True)

In [37]:
customer_profiles 

,CustomerID,total_spent,total_quantity,avg_price,category_counts,region_Asia,region_Europe,region_North America,region_South America
0,C0001,3354.52,12,278.334000,"{'Electronics': 3, 'Books': 1, 'Home Decor': 1}",0.0,0.0,0.0,1.0
1,C0002,1862.74,10,208.920000,"{'Home Decor': 2, 'Clothing': 2}",1.0,0.0,0.0,0.0
2,C0003,2725.38,14,195.707500,"{'Home Decor': 2, 'Clothing': 1, 'Electronics'...",0.0,0.0,0.0,1.0
3,C0004,5354.88,23,240.636250,"{'Books': 3, 'Home Decor': 3, 'Electronics': 2}",0.0,0.0,0.0,1.0
4,C0005,2034.24,7,291.603333,"{'Electronics': 2, 'Home Decor': 1}",1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
194,C0196,4982.88,12,416.992500,"{'Home Decor': 2, 'Books': 1, 'Clothing': 1}",0.0,1.0,0.0,0.0
195,C0197,1928.65,9,227.056667,"{'Electronics': 2, 'Home Decor': 1}",0.0,1.0,0.0,0.0
196,C0198,931.83,3,239.705000,"{'Electronics': 1, 'Clothing': 1}",0.0,1.0,0.0,0.0
197,C0199,1979.28,9,250.610000,"{'Electronics': 2, 'Home Decor': 2}",0.0,1.0,0.0,0.0


In [38]:
all_categories = set(
    category for counts in customer_profiles["category_counts"] for category in counts
)
for category in all_categories:
    customer_profiles[f"category_{category}"] = customer_profiles["category_counts"].apply(
        lambda x: x.get(category, 0)
    )

In [39]:
customer_profiles.drop(columns="category_counts", inplace=True)

In [40]:
customer_profiles

,CustomerID,total_spent,total_quantity,avg_price,region_Asia,region_Europe,region_North America,region_South America,category_Clothing,category_Books,category_Home Decor,category_Electronics
0,C0001,3354.52,12,278.334000,0.0,0.0,0.0,1.0,0,1,1,3
1,C0002,1862.74,10,208.920000,1.0,0.0,0.0,0.0,2,0,2,0
2,C0003,2725.38,14,195.707500,0.0,0.0,0.0,1.0,1,0,2,1
3,C0004,5354.88,23,240.636250,0.0,0.0,0.0,1.0,0,3,3,2
4,C0005,2034.24,7,291.603333,1.0,0.0,0.0,0.0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
194,C0196,4982.88,12,416.992500,0.0,1.0,0.0,0.0,1,1,2,0
195,C0197,1928.65,9,227.056667,0.0,1.0,0.0,0.0,0,0,1,2
196,C0198,931.83,3,239.705000,0.0,1.0,0.0,0.0,1,0,0,1
197,C0199,1979.28,9,250.610000,0.0,1.0,0.0,0.0,0,0,2,2


Standardize numerical features

In [41]:
scaler = StandardScaler()
numerical_features = ["total_spent", "total_quantity", "avg_price"]
scaled_features = scaler.fit_transform(customer_profiles[numerical_features])
scaled_features_df = pd.DataFrame(
    scaled_features, columns=[f"scaled_{col}" for col in numerical_features]
)

In [42]:
scaled_features_df

,scaled_total_spent,scaled_total_quantity,scaled_avg_price
0,-0.061701,-0.122033,0.094670
1,-0.877744,-0.448000,-0.904016
2,-0.405857,0.203934,-1.094109
3,1.032547,1.670787,-0.447702
4,-0.783929,-0.936951,0.285581
...,...,...,...
194,0.829053,-0.122033,2.089604
195,-0.841689,-0.610984,-0.643077
196,-1.386975,-1.588886,-0.461100
197,-0.813993,-0.610984,-0.304206


In [43]:
customer_profiles = pd.concat([customer_profiles, scaled_features_df], axis=1)

In [44]:
customer_profiles 

,CustomerID,total_spent,total_quantity,avg_price,region_Asia,region_Europe,region_North America,region_South America,category_Clothing,category_Books,category_Home Decor,category_Electronics,scaled_total_spent,scaled_total_quantity,scaled_avg_price
0,C0001,3354.52,12,278.334000,0.0,0.0,0.0,1.0,0,1,1,3,-0.061701,-0.122033,0.094670
1,C0002,1862.74,10,208.920000,1.0,0.0,0.0,0.0,2,0,2,0,-0.877744,-0.448000,-0.904016
2,C0003,2725.38,14,195.707500,0.0,0.0,0.0,1.0,1,0,2,1,-0.405857,0.203934,-1.094109
3,C0004,5354.88,23,240.636250,0.0,0.0,0.0,1.0,0,3,3,2,1.032547,1.670787,-0.447702
4,C0005,2034.24,7,291.603333,1.0,0.0,0.0,0.0,0,0,1,2,-0.783929,-0.936951,0.285581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,C0196,4982.88,12,416.992500,0.0,1.0,0.0,0.0,1,1,2,0,0.829053,-0.122033,2.089604
195,C0197,1928.65,9,227.056667,0.0,1.0,0.0,0.0,0,0,1,2,-0.841689,-0.610984,-0.643077
196,C0198,931.83,3,239.705000,0.0,1.0,0.0,0.0,1,0,0,1,-1.386975,-1.588886,-0.461100
197,C0199,1979.28,9,250.610000,0.0,1.0,0.0,0.0,0,0,2,2,-0.813993,-0.610984,-0.304206


In [45]:
customer_profiles.drop(columns=numerical_features, inplace=True)

In [46]:
customer_profiles

,CustomerID,region_Asia,region_Europe,region_North America,region_South America,category_Clothing,category_Books,category_Home Decor,category_Electronics,scaled_total_spent,scaled_total_quantity,scaled_avg_price
0,C0001,0.0,0.0,0.0,1.0,0,1,1,3,-0.061701,-0.122033,0.094670
1,C0002,1.0,0.0,0.0,0.0,2,0,2,0,-0.877744,-0.448000,-0.904016
2,C0003,0.0,0.0,0.0,1.0,1,0,2,1,-0.405857,0.203934,-1.094109
3,C0004,0.0,0.0,0.0,1.0,0,3,3,2,1.032547,1.670787,-0.447702
4,C0005,1.0,0.0,0.0,0.0,0,0,1,2,-0.783929,-0.936951,0.285581
...,...,...,...,...,...,...,...,...,...,...,...,...
194,C0196,0.0,1.0,0.0,0.0,1,1,2,0,0.829053,-0.122033,2.089604
195,C0197,0.0,1.0,0.0,0.0,0,0,1,2,-0.841689,-0.610984,-0.643077
196,C0198,0.0,1.0,0.0,0.0,1,0,0,1,-1.386975,-1.588886,-0.461100
197,C0199,0.0,1.0,0.0,0.0,0,0,2,2,-0.813993,-0.610984,-0.304206


Compute similarity using cosine similarity

In [47]:
feature_columns = [
    col for col in customer_profiles.columns if col not in ["CustomerID"]
]
customer_vectors = customer_profiles[feature_columns].values
similarity_matrix = cosine_similarity(customer_vectors)

Generate Lookalike recommendations

In [48]:
lookalike_map = {}
customer_ids = customer_profiles["CustomerID"].tolist()

for idx, customer_id in enumerate(customer_ids[:20]): 
    similarity_scores = similarity_matrix[idx]
    similar_customers = sorted(
        [(customer_ids[i], score) for i, score in enumerate(similarity_scores) if i != idx],
        key=lambda x: x[1],
        reverse=True
    )[:3]
    lookalike_map[customer_id] = similar_customers


Create Lookalike.csv

In [49]:
lookalike_list = []
for cust_id, similar_list in lookalike_map.items():
    formatted_similar_list = [(sc[0], round(sc[1], 2)) for sc in similar_list]
    lookalike_list.append({"cust_id": cust_id, "cust_id, score": formatted_similar_list})    

In [50]:
lookalike_df = pd.DataFrame(lookalike_list)

In [51]:
lookalike_df.to_csv("E:\STUDY\Project Zeotap/Alen_K_Johnson_Lookalike.csv", index=False)

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_26436\1716284089.py:1: SyntaxWarning: invalid escape sequence '\S'
  lookalike_df.to_csv("E:\STUDY\Project Zeotap/Alen_K_Johnson_Lookalike.csv", index=False)


In [52]:
lookalike_df 

,cust_id,"cust_id, score"
0,C0001,"[(C0120, 0.89), (C0127, 0.88), (C0069, 0.88)]"
1,C0002,"[(C0159, 0.94), (C0134, 0.93), (C0106, 0.92)]"
2,C0003,"[(C0031, 0.95), (C0195, 0.92), (C0039, 0.86)]"
3,C0004,"[(C0113, 0.96), (C0012, 0.93), (C0017, 0.92)]"
4,C0005,"[(C0007, 0.95), (C0140, 0.93), (C0146, 0.91)]"
5,C0006,"[(C0187, 0.95), (C0137, 0.87), (C0139, 0.85)]"
6,C0007,"[(C0005, 0.95), (C0146, 0.94), (C0115, 0.87)]"
7,C0008,"[(C0162, 0.94), (C0039, 0.92), (C0181, 0.91)]"
8,C0009,"[(C0198, 0.92), (C0061, 0.86), (C0150, 0.79)]"
9,C0010,"[(C0176, 0.89), (C0111, 0.87), (C0077, 0.86)]"
